# Diet Calculator

Alternatively, you can visit http://frepha.pythonanywhere.com/ to use the web application.

In [1]:
import pandas as pd
import numpy as np
from pulp import *
import re
import matplotlib
import matplotlib.pyplot as plt
from ortools.linear_solver import pywraplp

In [2]:
# change the directory to your own one
df = pd.read_excel("./NutrientsTest.xlsx")

In [3]:
veg = df['Category'] == "Vegetables"
fruit = df['Category'] == "Fruits"

df['Meat'] = df['Category'] == "Meat"
df['Meat'] = df['Meat'].astype(int)
df['FruitVeg'] = fruit | veg
df['FruitVeg'] = df['FruitVeg'].astype(int)

In [4]:
#select columns
columns3 = []
columns3.append("Category")
columns3.append("Food Name")
columns3.append("Price (RM)")
columns3.append("Fat, total (g/100g)")
columns3.append("Fibre, total dietary (g/100g)")
columns3.append("Protein, total; calculated from total nitrogen (g/100g)")
columns3.append("Sugars, total (g/100g)")
columns3.append("Fatty acids, total saturated (g/100g)")
columns3.append("Sugar, added")
columns3.append("Cholesterol (mg/100g)")
columns3.append("Energy (kcal/100g)")
columns3.append("Meat")
columns3.append("FruitVeg")
columns3.append("Vitamin C (mg/100g)")
columns3.append("Vitamin E, alpha-tocopherol equivalents (mg/100g)")
columns3.append("Calcium (mg/100g)")
columns3.append("Sodium (mg/100g)")
columns3.append("Iron (mg/100g)")
columns3.append("Zinc (mg/100g)")
columns3.append("Vitamin A, retinol equivalents (µg/100g)")
columns3.append("Vitamin D (μg/100g)")
columns3.append("Vitamin K (μg/100g)")
columns3.append("Manganese (μg/100g)")
columns3.append("Fat, total (g/100g)")
columns3.append("Fibre, total dietary (g/100g)")
columns3.append("Sodium (mg/100g)")
columns3

df2 = df[columns3]
data = df2.values.tolist()

In [5]:
def get_nutrient_constraints(gender):
    # Nutrient constraints for three categories.
    if (gender == 'male'):
        nutrients = [
            ["Fat, total (g/100g)", 73],
            ["Fibre, total dietary (g/100g)", 20],
            ["Protein, total; calculated from total nitrogen (g/100g)", 61],
            ["Sugars, total (g/100g)", 325],
            ["Fatty acids, total saturated (g/100g)", 20],
            ["Sugar, added", 50],
            ["Cholesterol (mg/100g)", 300],
            ["Energy (kcal/100g)", 2190],
            ["Meat", 1],
            ["FruitVeg", 4],
            ["Vitamin C (mg/100g)", 70],
            ["Vitamin E, alpha-tocopherol equivalents (mg/100g)", 10],
            ["Calcium (mg/100g)", 1000],
            ["Sodium (mg/100g)", 2300],
            ["Iron (mg/100g)", 9],
            ["Zinc (mg/100g)", 6.5],
            ["Vitamin A, retinol equivalents (µg/100g)", 600],
            ["Vitamin D (μg/100g)", 15], #!!!
            ["Vitamin K (μg/100g)", 65],
            ["Manganese (μg/100g)", 2300],
            ["Fat, total (g/100g)", 61],
            ["Fibre, total dietary (g/100g)", 70],
            ["Sodium (mg/100g)", 1500]
        ]
    elif (gender == 'female'):
        nutrients = [
            ["Fat, total (g/100g)", 63],
            ["Fibre, total dietary (g/100g)", 20],
            ["Protein, total; calculated from total nitrogen (g/100g)", 52],
            ["Sugars, total (g/100g)", 325],
            ["Fatty acids, total saturated (g/100g)", 20],
            ["Sugar, added", 50],
            ["Cholesterol (mg/100g)", 300],
            ["Energy (kcal/100g)", 1900],
            ["Meat", 1],
            ["FruitVeg", 4],
            ["Vitamin C (mg/100g)", 70],
            ["Vitamin E, alpha-tocopherol equivalents (mg/100g)", 7.5],
            ["Calcium (mg/100g)", 1000],
            ["Sodium (mg/100g)", 2300],
            ["Iron (mg/100g)", 20],
            ["Zinc (mg/100g)", 4.6],
            ["Vitamin A, retinol equivalents (µg/100g)", 600],
            ["Vitamin D (μg/100g)", 15], #!!!
            ["Vitamin K (μg/100g)", 55],
            ["Manganese (μg/100g)", 1800],
            ["Fat, total (g/100g)", 53],
            ["Fibre, total dietary (g/100g)", 70],
            ["Sodium (mg/100g)", 1500]
        ]
    elif (gender == 'children'):
        nutrients = [
            ["Fat, total (g/100g)", 68],
            ["Fibre, total dietary (g/100g)", 20],
            ["Protein, total; calculated from total nitrogen (g/100g)", 23],
            ["Sugars, total (g/100g)", 325],
            ["Fatty acids, total saturated (g/100g)", 20],
            ["Sugar, added", 50],
            ["Cholesterol (mg/100g)", 300],
            ["Energy (kcal/100g)", 1750],
            ["Meat", 1],
            ["FruitVeg", 4],
            ["Vitamin C (mg/100g)", 35],
            ["Vitamin E, alpha-tocopherol equivalents (mg/100g)", 7],
            ["Calcium (mg/100g)", 1000],
            ["Sodium (mg/100g)", 2300],
            ["Iron (mg/100g)", 6],
            ["Zinc (mg/100g)", 5.7],
            ["Vitamin A, retinol equivalents (µg/100g)", 500],
            ["Vitamin D (μg/100g)", 15], #!!!
            ["Vitamin K (μg/100g)", 25],
            ["Manganese (μg/100g)", 1500],
            ["Fat, total (g/100g)", 49],
            ["Fibre, total dietary (g/100g)", 70],
            ["Sodium (mg/100g)", 1200]
        ]
    return nutrients

In [6]:
def optimize_cost(gender):
    # Instantiate a Glop solver and naming it.
    solver = pywraplp.Solver('ZahirDiet', pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)
    
    # Declare an array to hold our variables.
    # 0 lower bound, 2 upper bound (max 300 gram per food)

    foods = [solver.NumVar(0, 3, item[1]) for item in data]
    print('Number of decision variables: ', solver.NumVariables())
    
    # index of constraints for inequality (>=)
    more = [ 1, 2, 7, 9, 10, 11, 12, 14, 
            15, 16, 17, 18, 19, 20, 22 ]

    nutrients = get_nutrient_constraints(gender)
    constraints = []
    df_food = ""
    df_nutrient = ""
    #print("Constraints:\n----------------------------")

    for i, nutrient in enumerate(nutrients):
        if (i in more):
            #print('{} >= {:.2f}'.format(nutrient[0], nutrient[1]))
            constraints.append(solver.Constraint(nutrient[1], solver.infinity()))
        else:
            #print('{} <= {:.2f}'.format(nutrient[0], nutrient[1]))
            constraints.append(solver.Constraint(-solver.infinity(), nutrient[1]))

        for j, item in enumerate(data):
            constraints[i].SetCoefficient(foods[j], item[i + 3])

    #print('----------------------------\nNumber of constraints =', solver.NumConstraints())\
    
    # Objective function: Minimize the sum of (price-normalized) foods.
    objective = solver.Objective()
    for i, food in enumerate(foods):
        objective.SetCoefficient(food, df2.loc[i]['Price (RM)'])
    objective.SetMinimization()

    status = solver.Solve()

    # Check that the problem has an optimal solution.
    if status != solver.OPTIMAL:
        print('The problem does not have an optimal solution!')
        if status == solver.FEASIBLE:
            print('A potentially suboptimal solution was found.')
        else:
            print('The solver could not solve the problem.')
            exit(1)

    #if there is optimal solution
    else:
        # Display the amounts (in dollars) to purchase of each food.
        nutrients_result = [0] * len(nutrients)
        price_table = [] #food list
        table = [] #nutrient list

        for i, food in enumerate(foods):
            if food.solution_value() > 0.0:
                price_table.append([data[i][1], food.solution_value()*100, food.solution_value()*data[i][2]])
                for j, _ in enumerate(nutrients):
                    nutrients_result[j] += data[i][j + 3] * food.solution_value()

        print('==============================\nDiet (Optimal) Price: RM{:.2f}\n=============================='.format(objective.Value()))

        for i, nutrient in enumerate(nutrients):
            if (i in more):
                table.append([nutrient[0], '≥ ' + str(nutrient[1]), nutrients_result[i]])
            else:
                table.append([nutrient[0], '≤ ' + str(nutrient[1]), nutrients_result[i]])

        column_names = ['Food', 'Gram', 'Price']
        df_food = pd.DataFrame(price_table, columns = column_names)
        df_food = df_food.sort_values(by = ['Food'], ascending=[True]).reset_index(drop=True)
        df_food = df_food.round(2)

        column_names = ['Nutrient', 'Constraint', 'Current Diet Value']
        df_nutrient = pd.DataFrame(table, columns = column_names)
        df_nutrient.loc[df_nutrient["Nutrient"] == "Fat, total (g/100g)", ["Nutrient"]] = "Total Fat (g)"
        df_nutrient.loc[df_nutrient["Nutrient"] == "Calcium (mg/100g)", ["Nutrient"]] = "Calcium (mg)"
        df_nutrient.loc[df_nutrient["Nutrient"] == "Cholesterol (mg/100g)", ["Nutrient"]] = "Cholesterol (mg)"
        df_nutrient.loc[df_nutrient["Nutrient"] == "Energy (kcal/100g)", ["Nutrient"]] = "Calories (kcal)"
        df_nutrient.loc[df_nutrient["Nutrient"] == "Fatty acids, total saturated (g/100g)", ["Nutrient"]] = "Total Sat. Fat (g)"
        df_nutrient.loc[df_nutrient["Nutrient"] == "Fibre, total dietary (g/100g)", ["Nutrient"]] = "Fibre (g)"
        df_nutrient.loc[df_nutrient["Nutrient"] == "FruitVeg", ["Nutrient"]] = "Fruits & Vegetables (g)"
        df_nutrient.loc[df_nutrient["Nutrient"] == "Meat", ["Nutrient"]] = "Meat (g)"
        df_nutrient.loc[df_nutrient["Nutrient"] == "Vitamin K (μg/100g)", ["Nutrient"]] = "Vitamin K (μg)"
        df_nutrient.loc[df_nutrient["Nutrient"] == "Vitamin E, alpha-tocopherol equivalents (mg/100g)", ["Nutrient"]] = "Vitamin E (mg)"
        df_nutrient.loc[df_nutrient["Nutrient"] == "Vitamin D (μg/100g)", ["Nutrient"]] = "Vitamin D (μg)"
        df_nutrient.loc[df_nutrient["Nutrient"] == "Iron (mg/100g)", ["Nutrient"]] = "Iron (mg)"
        df_nutrient.loc[df_nutrient["Nutrient"] == "Sodium (mg/100g)", ["Nutrient"]] = "Sodium (mg)"
        df_nutrient.loc[df_nutrient["Nutrient"] == "Sugar, added", ["Nutrient"]] = "Added Sugar (g)"
        df_nutrient.loc[df_nutrient["Nutrient"] == "Sugars, total (g/100g)", ["Nutrient"]] = "Carbohydrates (g)"
        df_nutrient.loc[df_nutrient["Nutrient"] == "Vitamin A, retinol equivalents (µg/100g)", ["Nutrient"]] = "Vitamin A (µg)"
        df_nutrient.loc[df_nutrient["Nutrient"] == "Vitamin C (mg/100g)", ["Nutrient"]] = "Vitamin C (mg)"
        df_nutrient.loc[df_nutrient["Nutrient"] == "Manganese (μg/100g)", ["Nutrient"]] = "Manganese (µg)"
        df_nutrient.loc[df_nutrient["Nutrient"] == "Zinc (mg/100g)", ["Nutrient"]] = "Zinc (mg)"
        df_nutrient.loc[df_nutrient["Nutrient"] == "Protein, total; calculated from total nitrogen (g/100g)", ["Nutrient"]] = "Protein (g)"
        df_nutrient = df_nutrient.sort_values(by = ['Nutrient', 'Constraint'], ascending=[True, False]).reset_index(drop=True)
        df_nutrient = df_nutrient.round(2)
    
    return df_food, df_nutrient

<hr>

### Diet for Male Adult
#### Diet Price

In [7]:
df_food, df_nutrient = optimize_cost('male')

Number of decision variables:  156
Diet (Optimal) Price: RM15.14


#### Diet Plan

In [8]:
df_food

,Food,Gram,Price
0,Baking powder,2.15,0.02
1,"Egg, chicken, white & yolk, raw",75.95,0.49
2,"Nut, almond, dry roasted, unblanched, no salt ...",55.86,2.23
3,"Oatmeal, dry",300.00,3.75
4,"Pineapple, flesh, raw",300.00,1.18
5,"Salt, table, iodised",2.62,0.05
6,"Spinach, leafy vegetable, raw, English",56.52,1.27
7,"Sugar, brown",49.00,0.19
8,"Watermelon, flesh, raw, combined varieties",43.48,0.17
9,"Yoghurt, Greek style, full-fat, composite",189.87,5.78


#### Diet Constraints

In [9]:
df_nutrient

,Nutrient,Constraint,Current Diet Value
0,Added Sugar (g),≤ 50,50.00
1,Calcium (mg),≥ 1000,1000.00
2,Calories (kcal),≥ 2190,2190.00
3,Carbohydrates (g),≤ 325,93.87
4,Cholesterol (mg),≤ 300,300.00
5,Fibre (g),≥ 20,45.83
6,Fibre (g),≤ 70,45.83
7,Fruits & Vegetables (g),≥ 4,4.00
8,Iron (mg),≥ 9,29.01
9,Manganese (µg),≥ 2300,20008.92


<hr>

### Diet for Female Adult
#### Diet Price

In [10]:
df_food, df_nutrient = optimize_cost('female')

Number of decision variables:  156
Diet (Optimal) Price: RM13.79


#### Diet Plan

In [11]:
df_food

,Food,Gram,Price
0,Baking powder,14.83,0.16
1,"Egg, chicken, white & yolk, raw",75.95,0.49
2,"Nut, almond, dry roasted, unblanched, no salt ...",25.42,1.02
3,"Oatmeal, dry",300.00,3.75
4,"Oil, olive",1.92,0.08
5,"Pineapple, flesh, raw",300.00,1.18
6,"Spinach, leafy vegetable, raw, English",47.83,1.08
7,"Sugar, brown",11.37,0.04
8,"Watermelon, flesh, raw, combined varieties",52.17,0.21
9,"Yoghurt, Greek style, full-fat, composite",189.87,5.78


#### Diet Constraints

In [12]:
df_nutrient

,Nutrient,Constraint,Current Diet Value
0,Added Sugar (g),≤ 50,50.00
1,Calcium (mg),≥ 1000,1000.00
2,Calories (kcal),≥ 1900,1900.00
3,Carbohydrates (g),≤ 325,57.26
4,Cholesterol (mg),≤ 300,300.00
5,Fibre (g),≥ 20,42.09
6,Fibre (g),≤ 70,42.09
7,Fruits & Vegetables (g),≥ 4,4.00
8,Iron (mg),≥ 20,26.83
9,Manganese (µg),≥ 1800,19136.68


<hr>

### Diet for Children
#### Diet Price

In [13]:
df_food, df_nutrient = optimize_cost('children')

Number of decision variables:  156
Diet (Optimal) Price: RM12.94


#### Diet Plan

In [14]:
df_food

,Food,Gram,Price
0,Baking powder,17.39,0.19
1,"Egg, chicken, white & yolk, raw",75.78,0.49
2,"Milk, cow, chocolate flavour, fluid, ultra-hig...",13.53,0.10
3,"Nut, almond, dry roasted, unblanched, no salt ...",22.55,0.90
4,"Oatmeal, dry",273.79,3.42
5,"Pineapple, flesh, raw",300.00,1.18
6,"Powder, curry",4.09,0.07
7,"Spinach, leafy vegetable, raw, English",21.74,0.49
8,"Watermelon, flesh, raw, combined varieties",78.26,0.31
9,"Yoghurt, Greek style, full-fat, composite",189.89,5.78


#### Diet Constraints

In [15]:
df_nutrient

,Nutrient,Constraint,Current Diet Value
0,Added Sugar (g),≤ 50,50.00
1,Calcium (mg),≥ 1000,1000.00
2,Calories (kcal),≥ 1750,1750.00
3,Carbohydrates (g),≤ 325,49.54
4,Cholesterol (mg),≤ 300,300.00
5,Fibre (g),≥ 20,39.67
6,Fibre (g),≤ 70,39.67
7,Fruits & Vegetables (g),≥ 4,4.00
8,Iron (mg),≥ 6,26.73
9,Manganese (µg),≥ 1500,17977.89
